<a href="https://colab.research.google.com/github/evdelph/MongoDB/blob/main/RedditVisualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import json
import nltk
import gensim
nltk.download('stopwords')
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english')) 
import plotly.express as px

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
! pip install --upgrade pip pyLDAvis

### Part 1: Import data

Load in data from scp file transfers in the form of pandas dataframes

In [2]:
posts = pd.read_csv('posts.csv')
posts.head(3)

,body,score
0,Just wanted to update with my personal experie...,23
1,"I’m currently on Emgality monthly, with topira...",24
2,Anyone have experience with a CGRP that was wo...,16


In [3]:
replies = pd.read_csv('replies.csv')
replies.head(3)

,replies,score
0,"[{""comment_1"":""That’s great. I hope it continu...",23
1,"[{""comment_1"":""As a nurse, Im sure this is inf...",24
2,"[{""comment_1"":""Yes. This is what happened to m...",16


### Part 2: Cleanse data
Convert 'replies' dataframe into a more useable format. Cleanse both datasets of special characters, whitespaces, and convert to lowercase.

In [4]:
# function to cleanse data
def cleanse_text(df,col):
  """
  Remove stopwords
  Remove special characters
  """
  spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–","\xa0","\xa0å","”","“"]

  for char in spec_chars:
      df[col] = df[col].str.replace(char, ' ').str.replace('  ',' ').str.lower().str.strip()

  doc = []

  for sentence in df[col]:
    sentence = sentence.split(" ")
    for word in sentence:
      if word not in stop_words:
        doc.append(word.strip())

  return doc

In [5]:
replies_cleansed = pd.DataFrame(columns=['Comments'])

comment_list = []
for reply in replies.replies:
  comments = json.loads(reply)
  for comment in comments:
    comment_list.append(" ".join(list(comment.values())))

replies_cleansed['Comments'] = comment_list
replies_cleansed.head(3)

,Comments
0,That’s great. I hope it continues to work so w...
1,"As a nurse, Im sure this is info youre well aw..."
2,Yes. This is what happened to me. Each of the ...


### Part 3: Perform NLTK on Unigrams
This will be used to analyze and visualize most frequent words from 1000 cgrpMigraine subreddit posts and comments.

In [6]:
# feed dataframes through text cleansing functions
post_doc = cleanse_text(posts,'body')
replies_doc = cleanse_text(replies_cleansed,'Comments')

# convert in dictionary form
docs = [post_doc,replies_doc]
dictionary = gensim.corpora.Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]

# compute frequencies
tfidf = gensim.models.TfidfModel(corpus)
post_bow = pd.Series({dictionary[id]:np.around(freq, decimals=2) for id, freq in tfidf[corpus[0]]}).to_frame()
replies_bow = pd.Series({dictionary[id]:np.around(freq, decimals=2) for id, freq in tfidf[corpus[1]]}).to_frame()

post_bow.columns = ['Frequency']
replies_bow.columns = ['Frequency']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.



In [7]:
# store top 50 most frequent words from comments
post_top_50 = post_bow.sort_values(by='Frequency',ascending=False).iloc[0:51,:]
post_top_50.head(3)

,Frequency
lupus,0.22
etc,0.19
https,0.19


In [8]:
# store top 50 most frequent words from replies
replies_top_50 = replies_bow.sort_values(by='Frequency',ascending=False).iloc[0:51,:]
replies_top_50.head(3)

,Frequency
belly,0.27
thigh,0.23
thank,0.23


### Part 4: Perform NLTK functions on bigrams
This will be used to analyze and visualize bigrams (pair of two words) from 1000 cgrpMigraine subreddit posts and comments.

In [ ]:
# compute bigrams on tokens
post_bigrams = list(nltk.bigrams(cleanse_text(posts,'body')))
replies_bigrams = list(nltk.bigrams(cleanse_text(replies_cleansed,'Comments')))

In [10]:
list(post_bigrams)[0:5]

[('wanted', 'update'),
 ('update', 'personal'),
 ('personal', 'experience'),
 ('experience', 'far'),
 ('far', 'i’d')]

In [11]:
list(replies_bigrams)[0:5]

[('that’s', 'great'),
 ('great', 'hope'),
 ('hope', 'continues'),
 ('continues', 'work'),
 ('work', 'well')]

In [12]:
# for every word, compute frequency distribution of bigrams
post_freq_dist = nltk.ConditionalFreqDist(post_bigrams)
replies_freq_dist = nltk.ConditionalFreqDist(replies_bigrams)

In [13]:
post_freq_dist

ConditionalFreqDist(nltk.probability.FreqDist,
                    {'09': FreqDist({'28eli': 1}),
                     '1': FreqDist({'12': 1, '800': 3, 'aimovig': 1}),
                     '10': FreqDist({'days': 3}),
                     '100mg': FreqDist({'abortive': 1}),
                     '11': FreqDist({'days': 1, 'months': 1}),
                     '12': FreqDist({'years': 1}),
                     '125mg': FreqDist({'daily': 1}),
                     '140': FreqDist({'might': 1}),
                     '140mg': FreqDist({'discontinuing': 1, 'fingers': 1}),
                     '15': FreqDist({'35': 1}),
                     '16': FreqDist({'previous': 1}),
                     '1605': FreqDist({'option': 1}),
                     '2': FreqDist({'3': 2, '5': 1, 'weeks': 1}),
                     '20': FreqDist({'30': 1, 'day': 1, 'days': 2}),
                     '2018': FreqDist({'migraines': 1}),
                     '22': FreqDist({'episodes': 1}),
                     '222s

In [14]:
replies_freq_dist

ConditionalFreqDist(nltk.probability.FreqDist,
                    {'that’s': FreqDist({'far': 1, 'great': 1, 'sharps': 1}),
                     'great': FreqDist({'consideration': 1,
                               'feeling': 1,
                               'hope': 1,
                               'info': 1,
                               'loading': 1,
                               'thank': 1,
                               'think': 1,
                               'time': 1}),
                     'hope': FreqDist({'aimovig': 1,
                               'continues': 1,
                               'doesnt': 1,
                               'emgality': 2,
                               'everything': 1,
                               'find': 1,
                               'long': 1,
                               'maybe': 1,
                               'med': 1,
                               'wasnt': 1,
                               'works': 1}),
                 

# Part 5: Visualize Most Frequent Unigrams
Visualize unigram word frequencies from posts and replies using plotly library.

In [ ]:
px.bar(post_top_50, x=post_top_50.index, y="Frequency",color="Frequency",title="Unigram Word Frequencies- Posts")

In [ ]:
px.bar(replies_top_50, x=replies_top_50.index, y="Frequency",color="Frequency",title="Unigram Word Frequencies- Replies").show()

### Part 6- Visualize Bigram Frequencies
The bigram frequency distribution stores all the word associations in the form of a nested dictionary. I will visualize some interesting key words to see what other words they are associated with.

There are three cgrp autoinjector medications: Aimovig, Emgality, and Ajovy. For every drug (word), I will visualize bigrams for each word, for both posts and replies.

#### Aimovig Post and Replies Bigrams

In [ ]:
# convert selected bigrams into dataframes
aimovig_post = pd.Series(dict(post_freq_dist['aimovig'])).to_frame()
aimovig_post.columns = ['Words']
px.bar(aimovig_post, x=aimovig_post.index, y='Words',color='Words',title="Bigrams- Aimovig Post").show()

In [ ]:
# convert selected bigrams into dataframes
aimovig_replies = pd.Series(dict(replies_freq_dist['aimovig'])).to_frame()
aimovig_replies.columns = ['Words']
px.bar(aimovig_replies, x=aimovig_replies.index, y='Words',color='Words',title="Bigrams- Aimovig Replies").show()

#### Emgality Post and Replies Bigrams

In [ ]:
# convert selected bigrams into dataframes
emgality_post = pd.Series(dict(post_freq_dist['emgality'])).to_frame()
emgality_post.columns = ['Words']
px.bar(emgality_post, x=emgality_post.index, y='Words',color='Words',title="Bigrams- Emgality Post").show()

In [ ]:
# convert selected bigrams into dataframes
emgality_replies = pd.Series(dict(replies_freq_dist['emgality'])).to_frame()
emgality_replies.columns = ['Words']
px.bar(emgality_replies, x=emgality_replies.index, y='Words',color='Words',title="Bigrams- Emgality Replies").show()

#### Ajovy Post and Replies Bigrams

In [ ]:
# convert selected bigrams into dataframes
ajovy_post = pd.Series(dict(post_freq_dist['ajovy'])).to_frame()
ajovy_post.columns = ['Words']
px.bar(ajovy_post, x=ajovy_post.index, y='Words',color='Words',title="Bigrams- Ajovy Post").show()

In [ ]:
# convert selected bigrams into dataframes
ajovy_replies = pd.Series(dict(replies_freq_dist['ajovy'])).to_frame()
ajovy_replies.columns = ['Words']
fig = px.bar(ajovy_replies, x=ajovy_replies.index, y='Words',color='Words',title="Bigrams- Ajovy Replies")
fig.show()

### Part 7: LDA Model Training- Posts

In [16]:
import gensim.corpora as corpora

id2word = corpora.Dictionary([post_doc])
texts = post_doc
corpus = [id2word.doc2bow(text.split()) for text in texts]
print(corpus)

[[(936, 1)], [(918, 1)], [(663, 1)], [(320, 1)], [(327, 1)], [(483, 1)], [(685, 1)], [(712, 1)], [(483, 1)], [(815, 1)], [(853, 1)], [(71, 1)], [(789, 1)], [(644, 1)], [(496, 1)], [(656, 1)], [(343, 1)], [(914, 1)], [(417, 1)], [(494, 1)], [(126, 1)], [(335, 1)], [(877, 1)], [(510, 1)], [(695, 1)], [(141, 1)], [(157, 1)], [(819, 1)], [(223, 1)], [(231, 1)], [(819, 1)], [(333, 1)], [(385, 1)], [(853, 1)], [(458, 1)], [(877, 1)], [(480, 1)], [(385, 1)], [(567, 1)], [(346, 1)], [(772, 1)], [(231, 1)], [(853, 1)], [(321, 1)], [(513, 1)], [(642, 1)], [(396, 1)], [(430, 1)], [(567, 1)], [(231, 1)], [(951, 1)], [(10, 1)], [(697, 1)], [(581, 1)], [(39, 1)], [(346, 1)], [(581, 1)], [(281, 1)], [(889, 1)], [(772, 1)], [(789, 1)], [(886, 1)], [(333, 1)], [(837, 1)], [(419, 1)], [(747, 1)], [(485, 1)], [(618, 1)], [(903, 1)], [(818, 1)], [(782, 1)], [(567, 1)], [(402, 1)], [(319, 1)], [(76, 1)], [(829, 1)], [(167, 1)], [(874, 1)], [(903, 1)], [(698, 1)], [(107, 1)], [(58, 1)], [(568, 1)], [(586, 1

In [ ]:
from pprint import pprint

num_topics = 10
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [34]:
import pickle

output = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)

with open('lda.pickle', 'wb') as handle:
    pickle.dump(output, handle)

In [37]:
import pyLDAvis
import pyLDAvis.gensim_models

with open('lda.pickle', 'rb') as handle:
    model = pickle.load(handle)

pyLDAvis.enable_notebook()
pyLDAvis.display(model)

### Part 8: LDA Topic Clustering- Replies

In [38]:
id2word2 = corpora.Dictionary([replies_doc])
texts2 = replies_doc
corpus2 = [id2word.doc2bow(text.split()) for text in texts2]
print(corpus)

[[(936, 1)], [(918, 1)], [(663, 1)], [(320, 1)], [(327, 1)], [(483, 1)], [(685, 1)], [(712, 1)], [(483, 1)], [(815, 1)], [(853, 1)], [(71, 1)], [(789, 1)], [(644, 1)], [(496, 1)], [(656, 1)], [(343, 1)], [(914, 1)], [(417, 1)], [(494, 1)], [(126, 1)], [(335, 1)], [(877, 1)], [(510, 1)], [(695, 1)], [(141, 1)], [(157, 1)], [(819, 1)], [(223, 1)], [(231, 1)], [(819, 1)], [(333, 1)], [(385, 1)], [(853, 1)], [(458, 1)], [(877, 1)], [(480, 1)], [(385, 1)], [(567, 1)], [(346, 1)], [(772, 1)], [(231, 1)], [(853, 1)], [(321, 1)], [(513, 1)], [(642, 1)], [(396, 1)], [(430, 1)], [(567, 1)], [(231, 1)], [(951, 1)], [(10, 1)], [(697, 1)], [(581, 1)], [(39, 1)], [(346, 1)], [(581, 1)], [(281, 1)], [(889, 1)], [(772, 1)], [(789, 1)], [(886, 1)], [(333, 1)], [(837, 1)], [(419, 1)], [(747, 1)], [(485, 1)], [(618, 1)], [(903, 1)], [(818, 1)], [(782, 1)], [(567, 1)], [(402, 1)], [(319, 1)], [(76, 1)], [(829, 1)], [(167, 1)], [(874, 1)], [(903, 1)], [(698, 1)], [(107, 1)], [(58, 1)], [(568, 1)], [(586, 1

In [ ]:
num_topics = 10
lda_model2 = gensim.models.LdaMulticore(corpus=corpus2,
                                       id2word=id2word2,
                                       num_topics=num_topics)

pprint(lda_model2.print_topics())
doc_lda2 = lda_model2[corpus]

In [40]:
output2 = pyLDAvis.gensim_models.prepare(lda_model2, corpus2, id2word2)

with open('lda_replies.pickle', 'wb') as handle:
    pickle.dump(output2, handle)

In [41]:
with open('lda_replies.pickle', 'rb') as handle:
    model2 = pickle.load(handle)

pyLDAvis.enable_notebook()
pyLDAvis.display(model2)

### Part 7: Brief Analysis of Results
The visualizations of the unigrams and bigrams indicate topics of interest on this subreddit. From the unigram analysis, we can see that words of interest include words involving symptoms and how to perform injections.

The replies show that the three medications are referenced with one another. Many migraineurs have tried many of these medications, and they share their experience with each one with respect to Aimovig. So there's a lot of comparison across the drugs.

Replies seem to offer more anecdotal text compared to the posts. From personal experience using this subreddit, you can receive many replies to confirm your own experience with the drug. As a result, replies will have more words and more variety compared to a given post.